# Patat : Plateforme Agnostique de Traitement et d'Analyse des Textes

## Notebook 6 : Importation Complete d'un Site

## Préliminaires

In [1]:
cd ../..

/Users/fm/Desktop/Work/Patat


In [2]:
import os
import shutil
import importlib
import urllib
import json
import pickle
import pandas as pd

## Choix du site

In [26]:
site = 'actu.fr'

In [27]:
site_url=f'https://{site}/'

## Lancement du crawler
## Consitution des listes d'urls :
- Liste d'url de listes d'articles
- Liste d'url d'articles

In [28]:
import patat.imp.parser_params
import patat.imp.parser
import patat.imp.crawler

In [29]:
importlib.reload(patat.imp.parser_params)
importlib.reload(patat.imp.parser)
importlib.reload(patat.imp.crawler)

<module 'patat.imp.crawler' from '/Users/fm/Desktop/Work/Patat/patat/imp/crawler.py'>

In [30]:
crawler = patat.imp.crawler.Crawler()

In [33]:
crawl_map = crawler.crawl_page(site_url)

ERROR:root:Importer.get_webpage:https://actu.fr/


Crawling : https://actu.fr/


### Crawling de l'ensemble des liens de listes d'articles

In [13]:
while True:
    visit_next =[]
    for url in crawler._crawl_map['list_links']:
        if url not in crawler._crawl_map['crawled_links']:
            visit_next.append(url)
    print(f'visit_next : {len(visit_next)}' )
    if len(visit_next)!=0:
        crawler.crawl_links(visit_next,max=10000)
    else:
        break

visit_next : 35
Crawling : https://www.francesoir.fr/rubrique/opinions
Crawling : https://www.francesoir.fr/rubrique/entretiens
Crawling : https://www.francesoir.fr/rubrique/bien-etre
Crawling : https://www.francesoir.fr/videos-l-entretien-essentiel
Crawling : https://www.francesoir.fr/rubrique/cinema
Crawling : https://www.francesoir.fr/rubrique/politique
Crawling : https://www.francesoir.fr/rubrique/videos-ne-pas-manquer
Crawling : https://www.francesoir.fr/rubrique/art-expo
Crawling : https://www.francesoir.fr/rubrique/celebrites
Crawling : https://www.francesoir.fr/rubrique/tourisme
Crawling : https://www.francesoir.fr/rubrique/sante
Crawling : https://www.francesoir.fr/rubrique/faits-divers
Crawling : https://www.francesoir.fr/rubrique/videos
Crawling : https://www.francesoir.fr/rubrique/lifestyle
Crawling : https://www.francesoir.fr/rubrique/economie
Crawling : https://www.francesoir.fr/rubrique/gastronomie
Crawling : https://www.francesoir.fr/rubrique/tribunes
Crawling : https:/

Crawling : https://www.francesoir.fr/rubrique/faits-divers?page=3
Crawling : https://www.francesoir.fr/rubrique/musique?page=3
Crawling : https://www.francesoir.fr/rubrique/opinions?page=3
Crawling : https://www.francesoir.fr/rubrique/tourisme?page=3
Crawling : https://www.francesoir.fr/rubrique/lifestyle?page=3
Crawling : https://www.francesoir.fr/rubrique/vie-quotidienne?page=3
Crawling : https://www.francesoir.fr/rubrique/sante?page=3
Crawling : https://www.francesoir.fr/rubrique/monde?page=3
Crawling : https://www.francesoir.fr/rubrique/emploi?page=3
Crawling : https://www.francesoir.fr/rubrique/france?page=3
Crawling : https://www.francesoir.fr/rubrique/politique?page=3
Crawling : https://www.francesoir.fr/rubrique/celebrites?page=3
Crawling : https://www.francesoir.fr/rubrique/culture?page=3
Crawling : https://www.francesoir.fr/rubrique/art-expo?page=3
Crawling : https://www.francesoir.fr/rubrique/cinema?page=3
Crawling : https://www.francesoir.fr/rubrique/economie?page=3
visit_n

Crawling : https://www.francesoir.fr/rubrique/lifestyle?page=8
Crawling : https://www.francesoir.fr/rubrique/environnement?page=8
Crawling : https://www.francesoir.fr/rubrique/faits-divers?page=8
Crawling : https://www.francesoir.fr/rubrique/societe?page=8
Crawling : https://www.francesoir.fr/rubrique/vie-quotidienne?page=8
Crawling : https://www.francesoir.fr/rubrique/medias?page=8
Crawling : https://www.francesoir.fr/rubrique/celebrites?page=8
Crawling : https://www.francesoir.fr/rubrique/transport?page=8
Crawling : https://www.francesoir.fr/rubrique/videos?page=8
Crawling : https://www.francesoir.fr/rubrique/opinions?page=8
Crawling : https://www.francesoir.fr/rubrique/bien-etre?page=8
Crawling : https://www.francesoir.fr/rubrique/france?page=8
Crawling : https://www.francesoir.fr/rubrique/science-tech?page=8
Crawling : https://www.francesoir.fr/rubrique/politique?page=8
Crawling : https://www.francesoir.fr/rubrique/monde?page=8
Crawling : https://www.francesoir.fr/rubrique/actualit

KeyboardInterrupt: 

In [14]:
len(crawler._crawl_map['list_links'])

350

# Importation et parsing des articles

In [15]:
article_urls = crawler._crawl_map['article_links']
len(article_urls)

11066

In [16]:
with open(f'data/tmp/220613-ArtcileUrls-{site}.pickle','wb') as file:
    pickle.dump(article_urls,file)
len(article_urls)

11066

In [17]:
with open(f'data/tmp/220613-ArtcileUrls-{site}.pickle','rb') as file:
    article_urls = pickle.load(file)
len(article_urls)

11066

In [18]:
def migrate(url):
    parsed = urllib.parse.urlparse(url)
    old_filename = parsed.path.replace('/', '-')+parsed.query.replace('/', '-')
    old_dir = parsed.netloc.replace('.', '-')
    old = 'data/html/'+old_dir+'/'+old_filename
    if os.path.isfile(old):
        new_filename = parsed.path.replace('/', '|')+parsed.query.replace('/', '|')
        new_dir = parsed.netloc
        shutil.copyfile('data/html/'+old_dir+'/'+old_filename, 
                        'data/html-v2/'+new_dir+'/'+new_filename)
        return True
    else:
        return False

In [19]:
parser = patat.imp.parser.Parser()

In [20]:
articles = []
counter = 0

In [21]:
for article_url in article_urls:
    counter += 1
    print(counter,article_url)
    if migrate(article_url):
        print('Migrée !')
    html = crawler.get_webpage(article_url)
    articles.append(parser.parse_article(article_url,html,site))
    if counter%100==0:
        df_articles=pd.DataFrame(articles)
        df_articles.to_csv(f'data/tmp/220613-CrawlArticles-{site}.csv')
df_articles=pd.DataFrame(articles)
df_articles.to_csv(f'data/tmp/220613-CrawlArticles-{site}.csv')


1 https://www.francesoir.fr/lifestyle-gastronomie/les-insectes-comestibles-de-demain-ont-deja-leur-usine
Migrée !
2 https://www.francesoir.fr/societe-economie/4-informations-judiciaires-ouvertes-contre-x-dans-la-gestion-de-lepidemie-olivier
Migrée !
3 https://www.francesoir.fr/afp-afp-france/la-frontiere-russe-de-la-finlande-lotan-accueillie-avec-soulagement
Migrée !
4 https://www.francesoir.fr/societe-science-tech/5g-huawei-en-france-cest-non
Migrée !
5 https://www.francesoir.fr/afp-afp-france/ukraine-la-position-dequilibriste-de-washington-de-plus-en-plus-difficile
Migrée !
6 https://www.francesoir.fr/opinions-entretiens/james-darlays-auteur-du-titre-dete-un-peu-de-vert-un-peu-de-brun-plagie-par-le
Migrée !
7 https://www.francesoir.fr/culture-cinema/le-palais-des-festivals-un-ecrin-vieillissant-aux-nombreux-liftings
Migrée !
8 https://www.francesoir.fr/lifestyle-bien-etre/le-cannabis-plus-nocif-pour-les-dents-que-la-cigarette
Migrée !
9 https://www.francesoir.fr/lifestyle-vie-quotidi

Migrée !
81 https://www.francesoir.fr/culture-cinema/deces-du-producteur-de-rambo-lamericano-hongrois-andy-vajna
Migrée !
82 https://www.francesoir.fr/societe-transport/le-gouvernement-annonce-une-prime-de-1000eu-pour-lachat-dune-voiture-electrique
Migrée !
83 https://www.francesoir.fr/culture-musique/les-francofolies-recoivent-supreme-ntm-joeystarr-et-kool-shen-sur-la-scene-du
Migrée !
84 https://www.francesoir.fr/afp-afp-france/euromillions-il-jouait-pour-creer-sa-fondation-cest-desormais-chose-faite
Migrée !
85 https://www.francesoir.fr/afp-afp-france/blinken-se-rend-en-israel-en-cisjordanie-au-maroc-et-en-algerie
Migrée !
86 https://www.francesoir.fr/societe-environnement/la-suede-affirme-avoir-trouve-une-maniere-sure-de-stocker-les-dechets
Migrée !
87 https://www.francesoir.fr/societe-transport/easy-jet-annonce-des-vols-100-electrique-info-ou-intox
Migrée !
88 https://www.francesoir.fr/opinions-tribunes/covid19-2020-2021-un-de-recherche-francaise-que-reste-t-il
Migrée !
89 https:/

Migrée !
149 https://www.francesoir.fr/societe-economie/la-fortune-des-dix-hommes-les-plus-riches-du-monde-double-depuis-le-debut-2020
Migrée !
150 https://www.francesoir.fr/culture-celebrites/bilal-hassani-critique-apres-une-video-polemique-sur-les-attentats
Migrée !
151 https://www.francesoir.fr/societe-transport-lifestyle-vie-quotidienne/greve-massive-la-ratp-ce-vendredi-13-les-previsions-de
Migrée !
152 https://www.francesoir.fr/lifestyle-tourisme/quand-un-fort-vauban-sur-une-ile-bretonne-est-loue-airbnb
Migrée !
153 https://www.francesoir.fr/lifestyle-bien-etre/non-boire-de-leau-naide-pas-lutter-contre-la-gueule-de-bois
Migrée !
154 https://www.francesoir.fr/societe-emploi/haute-normandie-pole-emploi-condamne-pour-harcelement-moral
Migrée !
155 https://www.francesoir.fr/afp-afp-france/dijon-ouvre-sa-cite-de-la-gastronomie-et-du-vin-ambassade-du-repas-francais-0
Migrée !
156 https://www.francesoir.fr/societe-faits-divers/sarcelles-discussions-batons-rompus-entre-lyceens-et-policier

216 https://www.francesoir.fr/culture-musique/eric-clapton-ne-se-produira-pas-dans-les-salles-ou-la-vaccination-est-requise
Migrée !
217 https://www.francesoir.fr/afp-afp-france/il-faudra-extraire-beaucoup-de-co2-de-lair-et-de-leau-selon-lonu
Migrée !
218 https://www.francesoir.fr/culture-art-expo/google-doodle-bks-iyengar-le-maitre-du-yoga-aurait-eu-97-ans-aujourdhui
219 https://www.francesoir.fr/societe-economie/arash-derambarsh-reste-avocat-camouflet-pour-luniversite-paris-la-sorbonne
Migrée !
220 https://www.francesoir.fr/culture-musique/il-y-46-ans-les-adieux-de-paris-josephine-baker
Migrée !
221 https://www.francesoir.fr/culture-art-expo/google-un-doodle-pour-vous-souhaiter-de-joyeuses-fetes
222 https://www.francesoir.fr/societe-environnement/descente-de-la-seine-la-nage-des-jeunes-souhaitent-sensibiliser-les-francais
Migrée !
223 https://www.francesoir.fr/politique-france/annonce-emmanuel-macron-candidat-reelection
Migrée !
224 https://www.francesoir.fr/societe-environnement/mia

288 https://www.francesoir.fr/opinions-tribunes-societe-science-tech/traitement-du-professeur-raoult-le-point-sur-les
Migrée !
289 https://www.francesoir.fr/afp-afp-france/unesco-un-dossier-pour-la-cote-de-granit-rose-dici-troisquatre-ans
Migrée !
290 https://www.francesoir.fr/culture-cinema/il-spoile-avengers-endgame-dans-le-cinema-et-se-fait-tabasser-par-des-fans
Migrée !
291 https://www.francesoir.fr/culture-musique/un-apres-les-fans-de-johnny-lui-rendent-hommage-la-madeleine
Migrée !
292 https://www.francesoir.fr/societe-economie/les-bottes-aigle-bientot-fabriquees-nouveau-en-france
Migrée !
293 https://www.francesoir.fr/culture-medias/cyril-dion-sentir-sur-mon-cou-le-souffle-et-lardeur-autour-de-ma-poitrine
Migrée !
294 https://www.francesoir.fr/afp-afp-france/un-mort-en-gare-de-valenciennes-apres-un-incendie-dans-un-wagon-de-fret
Migrée !
295 https://www.francesoir.fr/societe-economie/commerces-de-proximite-et-artisans-impactes-par-lannonce-de-la-disparition-des-aga
Migrée !
296 

...

Migrée !
10713 https://www.francesoir.fr/lifestyle-tourisme/chateaux-forts-de-france-ces-forteresses-visiter-diaporama
Migrée !
10714 https://www.francesoir.fr/lifestyle-gastronomie/la-vallee-du-rhone-veut-mettre-du-blanc-dans-son-vin
Migrée !
10715 https://www.francesoir.fr/societe-transport/avions-pollueurs-une-petite-eco-taxe-pour-quel-resultat
Migrée !
10716 https://www.francesoir.fr/lifestyle-bien-etre/au-japon-les-potions-miracles-anti-gueule-de-bois-font-fureur
Migrée !
10717 https://www.francesoir.fr/societe-faits-divers/tour-eiffel-refuse-de-briller-pour-macron
Migrée !
10718 https://www.francesoir.fr/opinions-tribunes/suivi-essai-clinique-moderna
Migrée !
10719 https://www.francesoir.fr/opinions-tribunes/un-monde-sous-ivermectine-essai-danticipation
Migrée !
10720 https://www.francesoir.fr/lifestyle-vie-quotidienne/fdj-resultat-euromillions-tirage-du-mardi-5-mai-2020
Migrée !
10721 https://www.francesoir.fr/societe-science-tech/coronavirus-et-le-tracking-numerique-devint-real

10788 https://www.francesoir.fr/societe-transport/greve-sncf-info-trafic-le-transilien-tres-perturbe-la-circulation-ligne-par-ligne
Migrée !
10789 https://www.francesoir.fr/societe-science-tech/ne-plus-avoir-peur-sur-les-reseaux-sociaux-grace-une-ia-qui-efface-les
Migrée !
10790 https://www.francesoir.fr/culture-musique/aliens-coldplay-devoile-le-dernier-titre-de-son-album-en-soutien-aux-migrants-video
10791 https://www.francesoir.fr/culture-musique/mamma-mia-revoila-abba
Migrée !
10792 https://www.francesoir.fr/societe-sante/jai-lhabitude-detre-pris-comme-bouc-emissaire-linterview-de-didier-raoult-sur-cnews
Migrée !
10793 https://www.francesoir.fr/culture-musique/britney-spears-en-plein-concert-un-desequilibre-monte-sur-scene-et-se-precipite
Migrée !
10794 https://www.francesoir.fr/culture-medias/notre-rapport-aux-medias-t-il-ete-modifie-par-le-confinement
Migrée !
10795 https://www.francesoir.fr/opinions-entretiens-societe-sante/interview-exclusive-mandeep-mehra-lhydroxychloroquine-p

10859 https://www.francesoir.fr/culture-art-expo/herge-au-grand-palais-le-pere-de-tintin-sexpose-sans-trop-se-devoiler
Migrée !
10860 https://www.francesoir.fr/transport/les-recettes-des-radars-routiers-en-chute-libre
Migrée !
10861 https://www.francesoir.fr/afp-afp-france/loligarque-russe-abramovitch-et-des-negociateurs-ukrainiens-ont-souffert-dun-possible
Migrée !
10862 https://www.francesoir.fr/lifestyle-tourisme/des-jeux-et-de-lentrainement-pour-les-animaux-du-zoo-de-paris
Migrée !
10863 https://www.francesoir.fr/politique-france/manifestations-en-france-ce-19-fevrier-alors-que-la-fin-du-passe-est-annoncee-dans
Migrée !
10864 https://www.francesoir.fr/societe-science-tech/combien-de-temps-passez-vous-sur-facebook-et-instagram
Migrée !
10865 https://www.francesoir.fr/lifestyle-gastronomie/un-americain-mange-son-30000e-big-mac-chez-mcdonalds-un-record
Migrée !
10866 https://www.francesoir.fr/afp-afp-france/macron-se-mue-enfin-en-president-candidat-en-pleine-crise-internationale
Migré

10929 https://www.francesoir.fr/societe-economie/dette-le-maire-defend-la-strategie-du-gouvernement-apres-lalerte-de-la-cour-des
Migrée !
10930 https://www.francesoir.fr/afp-afp-france/ukraine-marioupol-et-le-donbass-dans-le-viseur-des-forces-russes
Migrée !
10931 https://www.francesoir.fr/lifestyle-vie-quotidienne/resultats-du-loto-fdj-tirage-du-mercredi-24-juin-2020
Migrée !
10932 https://www.francesoir.fr/societe-emploi/coronavirus-le-chomage-partiel-sera-massif-mais-pris-en-charge-par-letat
Migrée !
10933 https://www.francesoir.fr/afp-afp-france/la-gauche-tente-de-se-rassembler-en-vue-des-legislatives-pour-contrer-macron
Migrée !
10934 https://www.francesoir.fr/lifestyle-bien-etre/chirurgie-lourde-comment-mieux-recuperer-et-raccourcir-la-convalescence
Migrée !
10935 https://www.francesoir.fr/lifestyle-gastronomie/une-marque-americaine-cree-une-glace-au-hot-dog
Migrée !
10936 https://www.francesoir.fr/societe-emploi/ouverture-dominicale-des-galeries-lafayette-haussmann-la-negociatio

11000 https://www.francesoir.fr/lifestyle-bien-etre/5-conseils-pour-tomber-enceinte
Migrée !
11001 https://www.francesoir.fr/societe-transport/cote-pollution-les-paquebots-ne-sont-pas-tout-beaux-mais-ca-se-soigne
Migrée !
11002 https://www.francesoir.fr/lifestyle-tourisme/tourisme-bienvenue-en-provence-occitane
Migrée !
11003 https://www.francesoir.fr/lifestyle-vie-quotidienne/halloween-la-pleine-lune-bleue-exceptionnelle-de-la-fete-le-31-octobre
Migrée !
11004 https://www.francesoir.fr/culture-livres/guerre-dalgerie-alain-vincenot-raconte-lhistoire-des-pieds-noirs
Migrée !
11005 https://www.francesoir.fr/opinions-editos/hold-oui-mais-lequel
Migrée !
11006 https://www.francesoir.fr/lifestyle-vie-quotidienne/resultat-pmu-le-quinte-de-ce-vendredi-26-juin-2020
Migrée !
11007 https://www.francesoir.fr/culture-medias/sait-enfin-qui-se-cache-derriere-loeuf-le-plus-dinstagram-photovideo
Migrée !
11008 https://www.francesoir.fr/culture-cinema/border-mystere-horreur-et-fantastique-la-suedoise-v

In [22]:
len(articles)

11066

In [23]:
df_articles = pd.DataFrame(articles)

In [24]:
df_articles.to_csv(f'data/corpus/220613-{site}.csv',index=False)

In [25]:
df_articles

,url,title,article,author,date,tag
0,https://www.francesoir.fr/lifestyle-gastronomi...,Les insectes comestibles de demain ont déjà le...,"Perdu au milieu des champs du Jura, ce bâtimen...",Auteur: \n Par Rebecca FRAS...,Publié le 13 mars 2018 - 09:03,[]
1,https://www.francesoir.fr/societe-economie/4-i...,"4 informations judiciaires ouvertes contre X, ...",Communiqué du parquet : 4 informations judicia...,Auteur(s)\nFranceSoir,Publié le 10 novembre 2020 - 17:11,[]
2,https://www.francesoir.fr/afp-afp-france/la-fr...,"A la frontière russe de la Finlande, l'Otan ac...",Perturbé depuis le début de la guerre en Ukrai...,Auteur: \n Par Elias HUUHTA...,Publié le 14 mai 2022 - 06:05,[]
3,https://www.francesoir.fr/societe-science-tech...,"5G : Huawei en France, c’est non",Orange espérait pouvoir réitérer en France ce ...,NaN,NaN,[]
4,https://www.francesoir.fr/afp-afp-france/ukrai...,Ukraine: la position d'équilibriste de Washing...,"La position d'équilibriste des Etats-Unis, qui...",Auteur: \n Par Sylvie LANTE...,Publié le 09 avril 2022 - 23:04,[]
...,...,...,...,...,...,...
11061,https://www.francesoir.fr/societe-transport/in...,Info trafic RATP et SNCF : fermetures des lign...,"Confinement oblige, la RATP et la SNCF ont réd...",Auteur(s)\nFrance-Soir,Publié le 26 mars 2020 - 19:03,[]
11062,https://www.francesoir.fr/culture-musique/azna...,Aznavour: un hommage en grand pour dire adieu ...,"""Ils sont venus, ils sont tous là..."" Personna...",Auteur: \n Par Aurélie MAYE...,Publié le 05 octobre 2018 - 06:10,[]
11063,https://www.francesoir.fr/societe-sante/covid-...,Covid-19: Sanofi publie des résultats positifs...,Le laboratoire français Sanofi a publié lundi ...,Auteur(s)\nPar AFP - Paris,Publié le 17 mai 2021 - 09:05,[]
11064,https://www.francesoir.fr/lifestyle-bien-etre/...,"A 110 ans passés, ""je tiens le coup"": vies de ...","Pour ses 118 ans vendredi, soeur André, l'une ...",Auteur: \n Par Sandra LAFFO...,Publié le 10 février 2022 - 07:02,[]
